# Download the needed file

In [ ]:
# -*- coding: utf-8 -*-
"""Face_Anomolyze.ipynb
Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1TBtViihvgzkntviMBScLIQY3UQ3EoAB2
"""

In [ ]:
"""# Run"""

!ls  ~/.insightface/models/
!gdown 1qXsQJ8ZT42_xSmWIYy85IcidpiZudOCB
!unzip buffalo_l.zip


!mkdir ~/.insightface
!mkdir ~/.insightface/models/
!cp -r buffalo_l  ~/.insightface/models/

!pip install insightface==0.7.1
!pip install onnxruntime

In [ ]:
output_path = "./processed_video"
! mkdir output_path

In [ ]:
# If buggy please commented out the following IPython magic to ensure Python compatibility.
# replace the writefile to your own face_analysis.py path

In [ ]:
# %%writefile /usr/local/lib/python3.10/dist-packages/insightface/app/face_analysis.py
# 
# from __future__ import division
# 
# import glob
# import os.path as osp
# 
# import numpy as np
# import onnxruntime
# from numpy.linalg import norm
# 
# from ..model_zoo import model_zoo
# from ..utils import DEFAULT_MP_NAME, ensure_available
# from .common import Face
# 
# __all__ = ['FaceAnalysis']
# 
# class FaceAnalysis:
#     def __init__(self, name=DEFAULT_MP_NAME, root='~/.insightface', allowed_modules=None, **kwargs):
#         onnxruntime.set_default_logger_severity(3)
#         self.models = {}
#         self.model_dir = ensure_available('models', name, root=root)
#         onnx_files = glob.glob(osp.join(self.model_dir, '*.onnx'))
#         onnx_files = sorted(onnx_files)
#         for onnx_file in onnx_files:
#             model = model_zoo.get_model(onnx_file, **kwargs)
#             if model is None:
#                 print('model not recognized:', onnx_file)
#             elif allowed_modules is not None and model.taskname not in allowed_modules:
#                 print('model ignore:', onnx_file, model.taskname)
#                 del model
#             elif model.taskname not in self.models and (allowed_modules is None or model.taskname in allowed_modules):
#                 print('find model:', onnx_file, model.taskname, model.input_shape, model.input_mean, model.input_std)
#                 self.models[model.taskname] = model
#             else:
#                 print('duplicated model task type, ignore:', onnx_file, model.taskname)
#                 del model
#         assert 'detection' in self.models
#         self.det_model = self.models['detection']
# 
# 
#     def prepare(self, ctx_id, det_thresh=0.5, det_size=(640, 640)):
#         self.det_thresh = det_thresh
#         assert det_size is not None
#         print('set det-size:', det_size)
#         self.det_size = det_size
#         for taskname, model in self.models.items():
#             if taskname=='detection':
#                 model.prepare(ctx_id, input_size=det_size, det_thresh=det_thresh)
#             else:
#                 model.prepare(ctx_id)
# 
#     def get(self, img, max_num=0):
#         bboxes, kpss = self.det_model.detect(img,
#                                              max_num=max_num,
#                                              metric='default')
#         if bboxes.shape[0] == 0:
#             return []
#         ret = []
#         for i in range(bboxes.shape[0]):
#             bbox = bboxes[i, 0:4]
#             det_score = bboxes[i, 4]
#             kps = None
#             if kpss is not None:
#                 kps = kpss[i]
#             face = Face(bbox=bbox, kps=kps, det_score=det_score)
#             for taskname, model in self.models.items():
#                 if taskname=='detection':
#                     continue
#                 model.get(img, face)
#             ret.append(face)
#         return ret
# 
#     def draw_on(self, img, faces):
#         import cv2
#         dimg = img.copy()
#         for i in range(len(faces)):
#             face = faces[i]
#             box = face.bbox.astype(np.int32)
#             color = (0, 0, 255)
#             cv2.rectangle(dimg, (box[0], box[1]), (box[2], box[3]), color, 2)
#             if face.kps is not None:
#                 kps = face.kps.astype(np.int32)
#                 #print(landmark.shape)
#                 for l in range(kps.shape[0]):
#                     color = (0, 0, 255)
#                     if l == 0 or l == 3:
#                         color = (0, 255, 0)
#                     cv2.circle(dimg, (kps[l][0], kps[l][1]), 1, color,
#                                2)
#             if face.gender is not None and face.age is not None:
#                 cv2.putText(dimg,'%s,%d'%(face.sex,face.age), (box[0]-1, box[1]-4),cv2.FONT_HERSHEY_COMPLEX,0.7,(0,255,0),1)
# 
#         return dimg

In [ ]:

# !wget https://master.dl.sourceforge.net/project/insightface.mirror/v0.7/scrfd_person_2.5g.onnx?viasf=1

import cv2
import numpy as np
import insightface

import imp
imp.reload(insightface)

from insightface.app import FaceAnalysis
from insightface.data import get_image as ins_get_image

# Method-1, use FaceAnalysis
# detector = insightface.model_zoo.get_model('scrfd_person_2.5g.onnx', download=True)
detector = FaceAnalysis(name="buffalo_l") # enable detection model only
detector.prepare(ctx_id=0,  det_thresh=0.4, det_size=(1280, 1280))

def detect_person(img, detector):
    bboxes, kpss = detector.detect(img, (680, 680))
    bboxes = np.round(bboxes[:,:4]).astype(np.int)
    kpss = np.round(kpss).astype(np.int)
    kpss[:,:,0] = np.clip(kpss[:,:,0], 0, img.shape[1])
    kpss[:,:,1] = np.clip(kpss[:,:,1], 0, img.shape[0])
    vbboxes = bboxes.copy()
    vbboxes[:,0] = kpss[:, 0, 0]
    vbboxes[:,1] = kpss[:, 0, 1]
    vbboxes[:,2] = kpss[:, 4, 0]
    vbboxes[:,3] = kpss[:, 4, 1]
    return bboxes, vbboxes

def get_detector_bbox(img):
  # bboxes, vbboxes = detect_person(img, detector)

  faces = detector.get(img)
  return faces


def blur_img(img, face, frame):
  bbox = face["bbox"]
  ksize = (50, 50)

  y1 = np.maximum(round(bbox[0]), 0)
  x1 = np.maximum(round(bbox[1]), 0)
  y2 = np.maximum(round(bbox[2]), 0)
  x2 = np.maximum(round(bbox[3]), 0)

  print(f"frame = {frame}, x1,x2,y1,y2", x1,x2,y1,y2, img.shape)

  # Using cv2.blur() method
  img[x1:x2, y1:y2] = cv2.blur(img[x1:x2, y1:y2], ksize)

  # Displaying the image
  return img

!mkdir /content/drive/MyDrive/saver_image_cvpr2024

import cv2
def cv_read_img(path):
  return  cv2.imread(path, 1)

import cv2, os


def rotate(img, mode=1):
# rotate(src, rotateCode[, dst]) ->
  if mode == 1:
    return cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
  elif mode == 2:
    return cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)
  else:
    print("mode=", mode)
    return img





def VideoProcess(video, mode=0):
    bboxs = []
    vid = cv2.VideoCapture(video)
    filename = video.split("/")[-1]
    save_file = f'/content/drive/MyDrive/saver_image_cvpr2024/{filename}.avi'
    if os.path.exists(save_file):
      return

    last_box = []
    last2_box = []
    last3_box = []

    print("processing: ", filename)
    if vid.isOpened():
        number=0
        while True:
            ret, frame = vid.read()
            if ret !=1:
               break

            if number == 0:
                fps = int(vid.get(cv2.CAP_PROP_FPS))
                h, w, c= frame.shape
                size = (w, h)
                saver = cv2.VideoWriter(save_file,
                         cv2.VideoWriter_fourcc(*'MJPG'),  fps, size)
            if mode !=0:
              frame = rotate(frame, mode)


            faces = get_detector_bbox(frame)
            bboxs.append(bboxs)
            for face in faces:
              frame = blur_img(frame, face, number)
              # last_box =

            for f in last_box:
               frame = blur_img(frame, f, number)

            for f in last2_box:
               frame = blur_img(frame, f, number)

            for f in last3_box:
               frame = blur_img(frame, f, number)

            last3_box = last2_box
            last2_box = last_box
            last_box = faces

            if mode !=0:
              frame = rotate(frame, 3 - mode)

            saver.write(frame)
            number+=1

    else:
        print('video reading failed!')

    print(f"save: {save_file} success")
    vid.release()
    saver.release()
    return bboxs


# Run for anomolyze video!

In [ ]:
# The mode is the rotation mode

v = "./media/media23.mp4"
VideoProcess(v, mode=1)

# Or

from glob import glob
videos = glob("./media/*.mp4")

for v in videos:
  VideoProcess(v)
